In [15]:
import io, requests
import pandas as pd, numpy as np
from PyPDF2 import PdfFileReader

def vaxScrape(date):
    print(f"checking vaccine data for {date:%A, %B %#d} ...")
    url = f"https://www.health.gov.au/sites/default/files/documents/{date:%Y}/{date:%m}/covid-19-vaccine-rollout-update-{date:%#d-%B-%Y}.pdf".lower()

    try:
        request = requests.get(url)
        file = io.BytesIO(request.content)
        reader = PdfFileReader(file)
        contents = reader.getPage(6).extractText().split("\n")
        
        try:                
            first_16 = float(contents[contents.index("dose 1") + 2].strip().replace(",", ""))
            second_16 = float(contents[contents.index("dose 2") + 2].strip().replace(",", ""))        
            first_12 = float(contents[len(contents) - contents[::-1].index("dose 1") + 1].strip().replace(",", ""))
            second_12 = float(contents[len(contents) - contents[::-1].index("dose 2") + 1].strip().replace(",", ""))

        except:
            first_16 = float(contents[contents.index("-dose 1") + 2].strip().replace(",", ""))
            second_16 = float(contents[contents.index("-dose 2") + 2].strip().replace(",", ""))
            first_12 = float(contents[len(contents) - contents[::-1].index("-dose 1") + 1].strip().replace(",", ""))
            second_12 = float(contents[len(contents) - contents[::-1].index("-dose 2") + 1].strip().replace(",", ""))

        first = first_16 + first_12
        second = second_16 + second_12
        
        contents = reader.getPage(2).extractText().split("\n")        
        boosters = float(contents[contents.index("Number of doses administered as part of the Commonwealth aged and disability care rollout") + 7].strip().replace(",", ""))
                
        vaxData.loc[0] = [first, second, boosters, date - pd.Timedelta(days=1)]        

    except:
        print(f"data unavailable for this date")
        date = date - pd.Timedelta(days=1)
        vaxScrape(date)

vaxData = pd.DataFrame(
    columns=["first", "second", "boosters", "date"]
)
        
today = pd.to_datetime(pd.Timestamp("now").date())
vaxScrape(today)

vaxData

checking vaccine data for Tuesday, January 11 ...
data unavailable for this date
checking vaccine data for Monday, January 10 ...


,first,second,boosters,date
0,367195.0,361923.0,79442.0,2022-01-09
